<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
#!pip install -q sqlalchemy ipython-sql prettytable #==1.3.9

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

In [5]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [6]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [8]:
%%sql
select distinct launch_site from spacextable;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [9]:
%%sql
select * from spacextable where launch_site like 'CCA%' limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [10]:
%%sql
select sum(payload_mass__kg_) as Total_Paload_Mass_in_kg_by_NASA_CRS
from spacextable where customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


Total_Paload_Mass_in_kg_by_NASA_CRS
45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [11]:
%%sql
select round(avg(payload_mass__kg_),3) as Avg_Payload_Mass_in_kg_by_F9_v1_1
from spacextable where booster_version like '%F9 v1.1%'

 * sqlite:///my_data1.db
Done.


Avg_Payload_Mass_in_kg_by_F9_v1_1
2534.667


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [12]:
%%sql
select min(date) as Date_of_First_Success_Landing_Outcome
from spacextable where landing_outcome like '%Success%'

 * sqlite:///my_data1.db
Done.


Date_of_First_Success_Landing_Outcome
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [13]:
%%sql
select booster_version, mission_outcome, landing_outcome
from spacextable 
    where landing_outcome like '%Success%' and 
landing_outcome like '%drone ship%' and 
payload_mass__kg_ between 4000 and 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version,Mission_Outcome,Landing_Outcome
F9 FT B1022,Success,Success (drone ship)
F9 FT B1026,Success,Success (drone ship)
F9 FT B1021.2,Success,Success (drone ship)
F9 FT B1031.2,Success,Success (drone ship)


### Task 7




##### List the total number of successful and failure mission outcomes


In [14]:
%%sql
select 
    (select count(mission_outcome) 
    from spacextable where mission_outcome like '%Success%')
as Total_Success_Mission_Outcome,
    (select count(mission_outcome)
    from spacextable where mission_outcome not like '%Success%')
as Total_Failed_Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Total_Success_Mission_Outcome,Total_Failed_Mission_Outcome
100,1


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [15]:
%%sql
select booster_version, mission_outcome, landing_outcome 
from spacextable 
where payload_mass__kg_ = (select max(payload_mass__kg_) from spacextable)

 * sqlite:///my_data1.db
Done.


Booster_Version,Mission_Outcome,Landing_Outcome
F9 B5 B1048.4,Success,Success
F9 B5 B1049.4,Success,Success
F9 B5 B1051.3,Success,Success
F9 B5 B1056.4,Success,Failure
F9 B5 B1048.5,Success,Failure
F9 B5 B1051.4,Success,Success
F9 B5 B1049.5,Success,Success
F9 B5 B1060.2,Success,Success
F9 B5 B1058.3,Success,Success
F9 B5 B1051.6,Success,Success


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [16]:
%%sql
select 
case strftime('%m', date) 
    when '01' then 'January' 
    when '02' then 'Febuary' 
    when '03' then 'March' 
    when '04' then 'April' 
    when '05' then 'May' 
    when '06' then 'June' 
    when '07' then 'July' 
    when '08' then 'August' 
    when '09' then 'September' 
    when '10' then 'October' 
    when '11' then 'November' 
    when '12' then 'December' 
    else '' end as Month,
landing_outcome as Failed_Landing_Outcome,
booster_version, launch_site 
from spacextable where strftime('%Y', date) = '2015'
and landing_outcome like '%Failure%';

 * sqlite:///my_data1.db
Done.


Month,Failed_Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [18]:
%%sql
select landing_outcome, count(landing_outcome) as Landing_Outcome_Count, 
rank() over (order by count(landing_outcome) desc) as Rank 
from spacextable where strftime('%F', date) between '2010-06-04' and '2017-03-20' 
group by landing_outcome order by landing_outcome_count desc

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Landing_Outcome_Count,Rank
No attempt,10,1
Success (drone ship),5,2
Failure (drone ship),5,2
Success (ground pad),3,4
Controlled (ocean),3,4
Uncontrolled (ocean),2,6
Failure (parachute),2,6
Precluded (drone ship),1,8


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
